# Datenanalyse: Verbrechen in San Francisco

Für das folgende Tutorial zur Datenanalyse nutzen wir die Programmiersprache Python.
Python ist eine gut lesbare Programmiersprache und wird typischerweise in der Datenanalyse genutzt.

Auch die Programmiersprache R wird häufig zur Datenverarbeitung genutzt, sie unterscheidet sich allerdings stark von Phyton.

### 1. Importieren von Bibliotheken

Zuerst importieren wir einige Standart-Bibliotheken für die Datenanalyse. Sie stellen viele Funktionalitäten bereit um nicht alles von Grund auf programmieren zu müssen.

Methoden der Bibliotheken können über den zugewiesenen Name der Bibliothek, dahinter ein Punkt und im Anschluss den Mehtodenname z.B. pd.read_csv(...) aufgerufen werden.

In [ ]:
# Mit einer Raute zu Beginn der Zeile, können wir einen Kommentar schreiben,
# dies ist kein Programmcode und wird daher nicht ausgeführt

# numpy und pandas werden zur Datenverarbeitung genutzt
# matplotlib wird zur Visualisierung der Daten genutzt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Diese Bibliotheken nutzen wird um Uhrzeiten zu verarbeiten
from datetime import datetime
import collections

### 2. Laden des Datensatzes

Wir laden den Datensatz, der als CSV-Datei (ähnlich einer Excel-Tabelle) vorliegt, als Pandas Dataframe um mit den Daten arbeiten zu können.

In [ ]:
# Auflisten der verfügbaren Datein
# mit print() können wir eine Textausgabe erzeugen
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Laden des Datensatzes train.csv.zip als Pandas Dataframe
crime_dataframe = pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')

### 3. Ersten Überblick über den Datensatz verschaffen

##### 3.1 Wie viele Einträge hat der Datensatz?

In [ ]:
# Die Methode len() liefert uns die Länge des Datensatzes
# f"" steht für formated string
# Ein String ist ein Datentyp der aus einer beliebigen Zeichenkette besteht
number_of_entries = len(crime_dataframe)
print(f"Dieser Datensatz hat {number_of_entries} Einträge!")

##### 3.2 In welche Kategorien ist der Datensatz aufgeteilt?

In [ ]:
# Die Methode columns() liefert uns die Spalten-Namen des Datensatzes
crime_columns = list(crime_dataframe.columns)
print(f"Dieser Datensatz hat die folgenden Spalten-Namen: \n {crime_columns}.")

##### 3.3 Head des Datensatzes
Um einschätzen zu können welche Werte wir für die einzelnen Kategorien zu erwarten haben, betrachten wir die "ersten Zeilen" des Datensatzes.


In [ ]:
# Den "Kopf" des Datensatz können wir mit der Methode head() anzeigen lassen
print(crime_dataframe.head())

### 4. Datenanlyse mit Diagrammen

Mit Diagrammen können wir uns noch einen besseren Überblick über die Daten verschaffen und erste Schlüsse ziehen.

#### 4.1 Das Histogramm
Wir wollen herausfinden welche Verbrechen am häufigsten begangen wurden und betrachten dazu die Kategorie 'Catagory'.

Ein Histogramm gibt eine Häufigkeitsverteilung an und ist deshalb gut geeignet.

In [ ]:
plt.hist(crime_dataframe['Category'], align='mid',bins=42)

# Formatierung des Plots
plt.xticks(rotation = 90)
fig = plt.gcf() 
fig.set_size_inches(11,8)

# Anzeigen des Plots
fig.show()

Als nächstes möchten wir wissen, ob es Wochentage gibt an denen besonders viele Verbrechen beganngen werden.

Dazu erstellen wir ein weiteres Histogramm und ersetzen die Kategorie mit 'DayOfWeek'.

In [ ]:
plt.hist(crime_dataframe['DayOfWeek'],bins=7, rwidth=0.9)

# Formatierung des Plots
plt.xticks(rotation = 90)
fig = plt.gcf() 
fig.set_size_inches(10,8)

# Anzeigen des Plots
fig.show()

An welchem Tag wird besonders oft ein Ladendiebstahl begangen?

Dazu kreieren wir ein Subset des Datensatzes und benutzten nur die Daten für die gilt, dass die 'Category' gleich einen bestimmten Wert, hier 'LARCENY/THEFT' ist.

In [ ]:
# Erstellen des Subsets
# Setze hier verschiedene Kategorien ein, um das Histogramm für diese anzeigen zu lassen
theft_subset = crime_dataframe[crime_dataframe['Category']=='LARCENY/THEFT']

# Anzeigen der ersten Spalten des Subsets
print(theft_subset.head())

plt.hist(theft_subset['DayOfWeek'], rwidth=0.9, color='red')

plt.xticks(rotation = 90)
fig = plt.gcf() 
fig.set_size_inches(11,8)
plt.show()

#### 4.2 Die Kreuztabelle (Cross Table)


Um Zusammenhänge zwischen mehrern Parametern erkennen zu können ist ist Kreuztabelle nützlich.
Sie stellt 2 Kategorien gegenüber und gibt die absoluten Häufigkeiten der Merkmale an.

Probiere jeweils zwei beliebige Spalten aus und bringe sie über die Kreuztabelle in Verbindung, 
Z.B. 'Category' und 'Resolution' um herauszufinden mit welcher Strafe man bei welchem Verbrechen zu rechnen hat.

Falls du vergessen hast, welche Spalten es gab, erinnerst du dich, wie wir uns diese anzeigen konnten?

In [ ]:
pd.crosstab(crime_dataframe['Category'], crime_dataframe['Resolution'])

#### 4.3 Das Kreisdiagramm (Pie Chart)

Das Kreisdiagramm kommt in der Regel nur selten zum Einsatz.

Wir machen uns das Kreisdiagramm aber hier zur Nutze um folgende Frage zu beantworten:

Werden nachts mehr Verbrechen begangen als tagsüber?

In [ ]:
# Die Daten zur Uhrzeit sind in der Spalte 'Dates' gespeichert,
# diese wählen wir aus
all_dates = crime_dataframe['Dates']

# Die Daten sind als String gespeichert, um an die Stunde zu kommen,
# möchten wir sie in ein Datumsformat "Datetime" umwandeln
print(f"Der Datentyp der Elemente in 'Dates' ist {type(all_dates[0])}!")
all_dates = [datetime.strptime(item, '%Y-%m-%d %H:%M:%S') for item in all_dates]

# Wir teilen die Uhrzeiten in Stunden auf, um dann zu sehen in welcher Stunde die meisten
# Delikte passieren
# mit .hour wählen wir die Stunde aus
# mit .minute kann man auch die Minuten auswählen
print(all_dates[0])
print(all_dates[0].hour)
all_hours = [item.hour for item in all_dates]
occurrences = collections.Counter(all_hours)
print(occurrences)

In [ ]:
labels =[]
sizes = []
explode = [0.2] * 24

for key,value in occurrences.items():
    labels.append(f"{key} Uhr")
    sizes.append(value)

fig1, ax1 = plt.subplots()
fig1.set_size_inches(6,6)
ax1.pie(sizes, labels=labels, explode=explode, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal') 
plt.show()